In [49]:
import pandas as pd
import boto3
from io import StringIO
import warnings
warnings.filterwarnings("ignore")

In [36]:
s3 = boto3.resource('s3')

bucket = s3.Bucket('xetra-1234')

bucket_obj1 = bucket.objects.filter(Prefix='dataset/2021-03-15/')
bucket_obj2 = bucket.objects.filter(Prefix='dataset/2021-03-16/')

desired_dates = ['2022-01-03', '2022-01-04']


In [26]:
objects = [obj for obj in bucket.objects.all()]
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-03/2022-01-03_BINS_XETR10.csv'),
 s3.Object

In [31]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

# Read multiple files

In [50]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in bucket.objects.all():
    for date in desired_dates:
        if date in obj.key:
            csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
            data = StringIO(csv_obj)
            df = pd.read_csv(data, delimiter=',')
            df_all = pd.concat([df_all, df], ignore_index=True, sort=False).dropna(axis=1, how='all')
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414,2
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649,3
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011,36
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308,22
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239075,DE0007164600,SAP,SAP SE O.N.,Common stock,EUR,2505077,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000,3
239076,MT0000580101,M8G,MEDIA AND GAMES INV. EO 1,Common stock,EUR,2509636,2022-01-04,16:44,4.118,4.118,4.118,4.118,24,2
239077,DE000A0Z23Q5,ADN1,ADESSO SE INH O.N.,Common stock,EUR,2504440,2022-01-04,16:45,196.800,196.800,196.800,196.800,7,2
239078,DE000A3E5ES0,BIO0,BIOTEST AG ST O.N.Z.VERK.,Common stock,EUR,6973997,2022-01-04,20:30,43.000,43.000,43.000,43.000,0,1


In [51]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897
...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0


In [52]:
df_all.dropna(inplace=True)
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-01-03,08:00,14.760,14.760,14.750,14.750,4414
1,DE000A0DJ6J9,2022-01-03,08:00,37.640,37.660,37.600,37.660,1649
2,DE000A0D6554,2022-01-03,08:00,13.990,14.030,13.940,13.960,23011
3,DE000A0D9PT0,2022-01-03,08:00,180.000,180.050,179.500,179.500,2308
4,DE000A0HN5C6,2022-01-03,08:00,37.280,37.280,37.280,37.280,2897
...,...,...,...,...,...,...,...,...
239075,DE0007164600,2022-01-04,16:44,124.380,124.380,124.380,124.380,4000
239076,MT0000580101,2022-01-04,16:44,4.118,4.118,4.118,4.118,24
239077,DE000A0Z23Q5,2022-01-04,16:45,196.800,196.800,196.800,196.800,7
239078,DE000A3E5ES0,2022-01-04,20:30,43.000,43.000,43.000,43.000,0


# Transformations